In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import mibian as mb
from datetime import datetime, timedelta

In [12]:
stock_symbol = 'GOOG'

# Fetch stock data
stock = yf.Ticker(stock_symbol)

# Fetch option chain
option_chain = stock.option_chain(stock.options[0])
calls = option_chain.calls
calls['cp'] = 'c'
puts = option_chain.puts
puts['cp'] = 'p'
options = pd.concat([calls, puts], axis=0)

In [24]:
def calculate_greeks_and_payoff(option, stock_price, interest_rate, days_to_expiry):
    call_flag = option['contractSymbol'].find('C') >= 0
    bs = mb.BS([stock_price, option['strike'], interest_rate, days_to_expiry], option['impliedVolatility'])

    if call_flag:
        delta = bs.callDelta
        gamma = bs.gamma
        theta = bs.callTheta
        vega = bs.vega
    else:
        delta = bs.putDelta
        gamma = bs.gamma
        theta = bs.putTheta
        vega = bs.vega

    payoff = option['lastPrice'] * delta

    return pd.Series({'payoff': payoff, 'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega})

In [32]:
expirations = stock.options
today = datetime.now()
days_to_expiry1 = max((datetime.fromisoformat(expirations[0]) - today).days, 1)
days_to_expiry2 = (datetime.fromisoformat(expirations[1]) - today).days

In [33]:
interest_rate = 0.01  # Example interest rate, you can use any appropriate source
stock_info = stock.info
options_with_greeks = options.apply(calculate_greeks_and_payoff, axis=1, stock_price=stock_info['open'],
                                    interest_rate=interest_rate, days_to_expiry=days_to_expiry1)
options_with_greeks.index = options['strike']
options_with_greeks

104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
104.65 1
1

,payoff,delta,gamma,theta,vega
strike,,,,,
45.0,61.55,1.0,0.0,-0.000012,0.0
50.0,45.00,1.0,0.0,-0.000014,0.0
55.0,51.24,1.0,0.0,-0.000015,0.0
60.0,46.75,1.0,0.0,-0.000016,0.0
65.0,40.53,1.0,0.0,-0.000018,0.0
...,...,...,...,...,...
135.0,-44.95,-1.0,0.0,0.000037,0.0
140.0,-46.00,-1.0,0.0,0.000038,0.0
145.0,-48.40,-1.0,0.0,0.000040,0.0


In [23]:
options

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,cp
0,GOOG230421C00045000,2023-04-17 19:53:39+00:00,45.0,61.55,60.35,61.40,0.000000,0.000000,1.0,14,9.226567,True,REGULAR,USD,c
1,GOOG230421C00050000,2023-03-15 16:15:04+00:00,50.0,45.00,56.90,59.85,0.000000,0.000000,1.0,64,11.953128,True,REGULAR,USD,c
2,GOOG230421C00055000,2023-04-17 15:00:57+00:00,55.0,51.24,50.35,51.40,0.000000,0.000000,1.0,6,7.281251,True,REGULAR,USD,c
3,GOOG230421C00060000,2023-04-10 15:10:50+00:00,60.0,46.75,45.35,46.40,0.000000,0.000000,70.0,129,6.429689,True,REGULAR,USD,c
4,GOOG230421C00065000,2023-04-20 15:26:47+00:00,65.0,40.53,40.35,41.40,2.229999,5.822453,2.0,26,5.648440,True,REGULAR,USD,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,GOOG230421P00135000,2023-02-27 20:48:24+00:00,135.0,44.95,30.50,35.05,0.000000,0.000000,6.0,0,5.423831,True,REGULAR,USD,p
51,GOOG230421P00140000,2023-03-15 13:38:48+00:00,140.0,46.00,30.15,34.45,0.000000,0.000000,2.0,0,3.113283,True,REGULAR,USD,p
52,GOOG230421P00145000,2022-11-21 18:45:29+00:00,145.0,48.40,53.70,56.45,0.000000,0.000000,2.0,0,12.444338,True,REGULAR,USD,p
53,GOOG230421P00155000,2023-04-18 19:54:40+00:00,155.0,49.75,48.60,49.65,0.000000,0.000000,NaN,0,2.781253,True,REGULAR,USD,p
